In [ ]:
import parsing

demo_file_name = r"D:/Samihan/Documents/Programming/Python Projects/CSGO Demo Parsing/demos/cloud9-vs-outsiders-m1-dust2.dem"
print(f"Parsing {demo_file_name}")
data = parsing.parse_demo_file(demo_file_name)
# print(data.keys())

In [1]:
# Loading demo from .json
import models
import json

with open("cloud9-vs-outsiders-m1-dust2.json", "r") as file:
# with open("cloud9-vs-outsiders-m2-ancient.json", "r") as file:
    data = json.load(file)

demo = models.deserialize_demo_data(data)

# Creating timeline and events from demo
import timelining

timeline = timelining.create_timeline(demo)
# print(len(timeline.events))
# for event in timeline.events:
#     print(event)

d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\timelining.py:1057: RuntimeWarning: invalid value encountered in double_scalars
  degrees_apart = 180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))/np.pi
d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\timelining.py:1057: RuntimeWarning: invalid value encountered in arccos
  degrees_apart = 180*np.arccos(np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2)))/np.pi


In [2]:
import timelining

round_boundary_ticks: list[int] = [(round.frames[0].tick, round.frames[-1].tick) for round in demo.game_rounds]

round_frame_ticks: list[list[int]] = []
for round in demo.game_rounds:
    round_frame_ticks.append([frame.tick for frame in round.frames])

organized_events: list[timelining.Event] = [[] for _ in range(0, len(demo.game_rounds))]
for event in timeline.events:
    for frame_index, frame_ticks in enumerate(round_frame_ticks):
        if event.tick in frame_ticks:
            organized_events[frame_index].append(event)
            break
# print(len(organized_events))
# print([len(frame) for frame in organized_events])

import plotting

round = demo.game_rounds[0]

plotting.plot_round(f"round_{round.round_number}", round, map_name=demo.map_name)

# import cProfile
# import pstats
# with cProfile.Profile() as pr:
#     plotting.plot_round(f"round_{round.round_number}", round, map_name=demo.map_name)

# stats = pstats.Stats(pr)
# stats.sort_stats(pstats.SortKey.TIME)
# stats.print_stats()
# stats.dump_stats(filename="info.prof")

# TODO: Organize events into frames so each frame of the GIF can have the associated events grouped with it

# TODO: Increase frame drawing speed (drawing view vectors is probably what takes the most time so maybe optimize that)
# TODO: Add controlled-area growth rules and have the areas expand as much as possible for each frame

Drawing frames: 100%|██████████| 96/96 [25:47<00:00, 16.12s/it]


         3068806469 function calls (3061403403 primitive calls) in 1556.559 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
383747840  807.884    0.000 1118.068    0.000 d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\venv\lib\site-packages\awpy\analytics\nav.py:9(point_in_area)
768107431  126.514    0.000  126.514    0.000 {built-in method builtins.min}
      817   97.192    0.119 1218.615    1.492 d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\mathing.py:24(trace_vision)
436110418   84.123    0.000   84.124    0.000 {built-in method builtins.max}
767839973   66.472    0.000   66.472    0.000 {method 'keys' of 'dict' objects}
387317747/387311580   35.254    0.000   35.260    0.000 {built-in method builtins.len}
  3077304   23.852    0.000   25.243    0.000 d:\Samihan\Documents\Programming\Python Projects\CSGO Demo Parsing\venv\lib\site-packages\matplotlib\transforms.py:207(set_childre

In [ ]:
# Writing events to csv

import csv
from dataclasses import fields

with open("events.csv", "w") as file:
    writer = csv.writer(file)
    all_fields: list[str] = []
    for event in timeline.events:
        all_fields.extend([field.name for field in fields(event)])
    unique_fields = list(dict.fromkeys(all_fields))
    # print(unique_fields)
    writer.writerow([field for field in unique_fields])
    for event in timeline.events:
        writer.writerow([getattr(event, field, None) for field in unique_fields])

# Writing event descriptions to .csv

import csv
from dataclasses import fields

with open("event_descriptions.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["tick", "name", "description"])
    for event in timeline.events:
        writer.writerow([event.tick, event.__class__.__name__, str(event)])